In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pybamm
import pandas as pd
import timeit
start = timeit.default_timer()
pybamm.set_logging_level("INFO")

In [2]:
model = pybamm.lithium_ion.DFN()

param = pybamm.ParameterValues("OKane2023")

param.update({"Upper voltage cut-off [V]": 4.5})
param.update({"Lower voltage cut-off [V]": 2.4})
Cycles = 10
experiment = pybamm.Experiment(
    [
        (
            "Discharge at 1C until 2.5V",
            "Charge at 0.3C until 4.2V (3 minute period)",
            "Hold at 4.2V until C/100 (3 minute period)"
        )
    ]
    * Cycles,
)

# define the simulation
sim = pybamm.Simulation(
    model,
    experiment=experiment,
    parameter_values=param,
    solver=pybamm.CasadiSolver()
    #solver=pybamm.CasadiSolver("fast with events")
)

solution = sim.solve(calc_esoh=False)
stop = timeit.default_timer()
print("running time: " + str(stop - start) + "s")
# plot

V = solution["Terminal voltage [V]"].entries
t_all = solution["Time [h]"].entries
dis_cap = solution["Discharge capacity [A.h]"].entries
#Cap_thro = solution["Throughput capacity [A.h]"].entries

plt.plot(t_all, dis_cap, 'b-', label="", linewidth=4)
plt.xlabel("Charge throughput [Ah]")
plt.ylabel("Discharge capacity [Ah]")
# plt.xticks(np.arange(0, 150, 10))
# plt.yticks(np.arange(0.0, 5.1, 1))
# plt.xlim([0, 150])
# plt.ylim([0.0, 5.1])
plt.legend(facecolor='white', framealpha=0)
plt.show()

2023-03-09 16:52:31.756 - [INFO] base_model._build_model(550): Start building Doyle-Fuller-Newman model


AttributeError: 'LithiumIonParameters' object has no attribute 'rho_cr'

In [ ]:
model = pybamm.lithium_ion.DFN()
param = pybamm.ParameterValues("Chen2020")
# Adjust SEI parameters
param.update({"Inner SEI lithium interstitial diffusivity [m2.s-1]": 3e-19})
# Override SEI thickness dependence of lithium plating
param.update({"Dead lithium decay rate [s-1]": 1e-6})
param.update({"Lithium plating kinetic rate constant [m.s-1]": 1e-10})
# Adjust mechanical degradation parameters
param.update({"Negative electrode cracking rate": 1e-22})
param.update({"Negative electrode LAM constant proportional term [s-1]": 1e-9})
param.update({"Positive electrode LAM constant proportional term [s-1]": 1e-9})
param.update({"Electrolyte conductivity [S.m-1]": electrolyte_conductivity_Valoen2005})
param.update({"Electrolyte diffusivity [m2.s-1]": electrolyte_diffusivity_Valoen2005})

In [ ]:
# define experiment
V_max = 4.2;        V_min = 2.5; 
Update_Cycles = 10; RPT_Cycles = 1;
charge_time_mins = 60 * 4.86491/5
exp_AGE_text = [(
    f"Charge at 5 C for {charge_time_mins} minutes",
    f"Discharge at 1 C until {V_min} V", 
    ),  ]
# step index for ageing
step_AGE_CD =0;   step_AGE_CC =1;   step_AGE_CV =2;

exp_RPT_text = [ (
    f"Charge at 1C until {V_max} V",  
    f"Hold at {V_max}V until C/100",
    f"Discharge at 0.1C until {V_min} V (5 minute period)",  
    "Rest for 1 hours (5 minute period)",  
    f"Charge at 0.1C until {V_max} V (5 minute period)",
    "Rest for 1 hours (5 minute period)",
    f"Discharge at 1C until {V_min} V",  
    f"Hold at {V_min} V until C/100",
    ) ]
# step index for RPT
step_RPT_CD = 2;  step_RPT_RE =3;   step_RPT_CC = 4;  
Experiment_Long   = pybamm.Experiment( exp_AGE_text * Update_Cycles  )  
Experiment_RPT    = pybamm.Experiment( exp_RPT_text * RPT_Cycles     ) 
Experiment_Breakin= pybamm.Experiment( exp_RPT_text * RPT_Cycles     )


In [ ]:
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 60,  # negative particle
    "r_p": 20,  # positive particle
}

In [ ]:
sim1 = pybamm.Simulation(
    model, 
    experiment=Experiment_Breakin,
    parameter_values=param,
    solver=pybamm.CasadiSolver(),
    var_pts=var_pts,
)
solution1 = sim1.solve(
    save_at_cycles=1
)

stop = timeit.default_timer()
print('running time: ' + str(stop - start) +'s')

In [ ]:
model_2 = model.set_initial_conditions_from(solution1, inplace=False)
sim2 = pybamm.Simulation(
    model_2, 
    experiment=Experiment_Long,
    parameter_values=param,
    solver=pybamm.CasadiSolver(),
    var_pts=var_pts,
)
solution2 = sim2.solve(
    save_at_cycles=[1,5,10] 
)

In [ ]:
# plot SOH, LAM, LII
# SOH must be defined ourselves? 

In [ ]:
label = ["Initial solution","Second solution"] 
var = [
    "Loss of lithium inventory, including electrolyte [%]",
    "LAM_ne [%]",
    "Loss of active material in negative electrode [%]",
    "LAM_pe [%]",
    "Loss of active material in positive electrode [%]",
    'LLI [%]', 
    'Throughput capacity [A.h]',
    "Discharge capacity [A.h]"
]
quick_plot = pybamm.QuickPlot(
    [solution1,solution2], var,label,variable_limits='tight') #     
quick_plot.dynamic_plot();

In [ ]:
print(solution2['Throughput capacity [A.h]'].entries[-1])
print(solution2.cycles[-1].steps[-1]['Throughput capacity [A.h]'].entries[-1])
print(solution2.cycles[-1].steps[1]['Throughput capacity [A.h]'].entries)

for step in solution2.cycles[-1].steps:
    step['Throughput capacity [A.h]'].entries += solution1['Throughput capacity [A.h]'].entries[-1]

In [ ]:
# need to do this every time next solution is got:
solution2['Throughput capacity [A.h]'].entries += solution1['Throughput capacity [A.h]'].entries[-1]
"Throughput energy [W.h]",

In [ ]:
plt.plot(solution1['Throughput capacity [A.h]'].entries)
plt.plot(solution2['Throughput capacity [A.h]'].entries)

In [ ]:
plt.plot(solution2['Cell temperature [K]'].entries )

In [ ]:
solution2['Cell temperature [C]'].entries 